In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV, cross_validate, cross_val_score
from sklearn.metrics import accuracy_score, recall_score
from scipy.stats import randint
from xgboost import XGBClassifier


XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/schoi330/miniconda3/envs/music/lib/python3.7/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib\n  Referenced from: /Users/schoi330/miniconda3/envs/music/lib/python3.7/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file)"]


In [2]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier

In [25]:
classifiers = [
    LogisticRegression(),
    SGDClassifier(random_state=3),
    SVC(kernel="rbf", C=0.025, probability=True),
    GaussianNB(),
    KNeighborsClassifier(3),
    RandomForestClassifier(),
    GradientBoostingClassifier(),
    AdaBoostClassifier(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis(),
    MLPClassifier()]


In [31]:
year = '2021'
df = pd.read_csv('years/'+year+'.csv')
df2 = pd.read_csv('random_sample/'+year+'.csv')


In [32]:
weeks = df.to_numpy()[:, -1]
percentile = [np.percentile(weeks, i*10) for i in range(1,10)]
percentile = np.percentile(weeks, 75)

In [34]:
df3 = df.append(df2)
Xy = df3.iloc[:,7:].to_numpy()
X = Xy[:, :-2]
y = Xy[:,-1]
top = percentile
print(top)
label = y.copy()
label[y <= 0] = 0
label[y > 1] = 1
y = label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

15.0


In [35]:
val, count = np.unique(y_test, return_counts= True)
count

array([205, 191])

In [36]:
for clf in classifiers:
    print("="*30)
    name = clf.__class__.__name__
    print(name)
    # clf.fit(X_train, y_train)
    
    
    # train_predictions = clf.predict(X_test)
    # acc = accuracy_score(y_test, train_predictions)
    acc = np.mean(cross_validate(clf, X, y)['test_score'])
    clf.fit(X_train, y_train)
    rec = recall_score(y_test,clf.predict(X_test))
    print("Recall: {:.4%}".format(rec))
    print("Accuracy: {:.4%}".format(acc))
    
print("="*30)

LogisticRegression
Recall: 66.4921%
Accuracy: 59.6212%
SGDClassifier
Recall: 100.0000%
Accuracy: 49.9242%
SVC
Recall: 100.0000%
Accuracy: 50.6061%
GaussianNB
Recall: 92.1466%
Accuracy: 64.0152%
KNeighborsClassifier
Recall: 53.4031%
Accuracy: 53.4091%
RandomForestClassifier
Recall: 75.9162%
Accuracy: 65.4545%
GradientBoostingClassifier
Recall: 78.0105%
Accuracy: 65.9848%
AdaBoostClassifier
Recall: 72.2513%
Accuracy: 65.5303%
LinearDiscriminantAnalysis
Recall: 87.4346%
Accuracy: 65.2273%
QuadraticDiscriminantAnalysis
Recall: 92.1466%
Accuracy: 65.3788%
MLPClassifier
Recall: 100.0000%
Accuracy: 49.9242%


In [13]:
df3 = df.append(df2)
Xy = df3.iloc[:,7:].to_numpy()
X = Xy[:, :-2]
y = Xy[:,-1]
percentile_eval = {}
for top in percentile:
    label = y.copy()
    label[y <= top] = 0
    label[y > top] = 1
    X_train, X_test, y_train, y_test = train_test_split(X, label, test_size=0.3)
    best_clf = None
    max_acc = 0
    for clf in classifiers:
        acc = np.mean(cross_validate(clf, X, y)['test_score'])
        if acc > max_acc:
            max_acc = acc
            best_clf = clf.__class__.__name__
    percentile_eval[top] = (best_clf, max_acc)
    

[1.0, 1.0, 1.0, 2.0, 4.0, 7.0, 12.0, 18.0, 22.399999999999977]


In [23]:
percentile_eval

{1.0: ('GaussianNB', 1.0),
 2.0: ('GaussianNB', 1.0),
 4.0: ('GaussianNB', 1.0),
 7.0: ('GaussianNB', 1.0),
 12.0: ('GaussianNB', 1.0),
 18.0: ('GaussianNB', 1.0),
 22.399999999999977: ('GaussianNB', 1.0)}